In [1]:
import faiss
import json
from transformers import AutoTokenizer, RagRetriever, RagTokenForGeneration

# Load FAISS index
index = faiss.read_index("legal_cases_index.faiss")

# Load metadata
with open("metadata.json", "r") as f:
    metadata = json.load(f)

# Load RAG model and tokenizer
rag_tokenizer = AutoTokenizer.from_pretrained("facebook/rag-sequence-nq")
rag_model = RagTokenForGeneration.from_pretrained("facebook/rag-sequence-nq")


2024-12-04 09:47:45.390201: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer, AutoModel

# Load the embedding model and tokenizer
embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")


In [11]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    embeddings = embedding_model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# functions
def retrieve_passages(query, top_k=5):
    query_embedding = embed_text(query)
    _, indices = index.search(query_embedding, k=top_k)
    return [metadata[idx]["text"] for idx in indices[0]]

query = "What is the ruling on minors' ability to enlist in the navy?"
passages = retrieve_passages(query, top_k=5)

# Debug: Print passages
print("\nTop Retrieved Passages:")
for i, passage in enumerate(passages):
    print(f"Passage {i+1}: {passage}")


# Debug: Check tokenization of a sample passage
sample_passage = passages[0]  # Use the first retrieved passage
tokenized_passage = rag_tokenizer(sample_passage, return_tensors="pt", padding=True, truncation=True)
print("\nTokenized Passage:")
print(tokenized_passage)


def generate_summary(query, passages):
    # Tokenize query
    input_ids = rag_tokenizer(query, return_tensors="pt").input_ids

    # Tokenize passages as context
    context_input_ids = []
    for passage in passages:
        context_input = rag_tokenizer(passage, return_tensors="pt", padding=True, truncation=True).input_ids
        context_input_ids.append(context_input)

    # Combine context input IDs
    context_input_ids = torch.cat(context_input_ids, dim=0)

    # Generate response using the model
    generated_ids = rag_model.generate(
        input_ids=input_ids,
        context_input_ids=context_input_ids
    )

    # Decode and return the generated text
    return rag_tokenizer.decode(generated_ids[0], skip_special_tokens=True)



Top Retrieved Passages:
Passage 1: ht of the parent, or other person irr loco parentis, to object to the enlistment of a minor in the navy, is clearly limited to cases where the minor is but 18 years of age. Jf he is above that age, and under 21 years of age, he can bind himself by enlisting until he attains his majority, at which ti
Passage 2: hority that enlistments in the navy, though, made without consent of the parent or guardian, are binding, and the minor cannot avoid them. See U. S. v. Bainbridge, 1 Mason, 71, Fed. Cas. No. 14,497; U. S. v. Blakeney, 3 Grat. 405.
But it is otherwise as to enlistments in the army. The distinction is
Passage 3:  statute since that time.
By an act of Congress approved May 12, 1879 (21 Stat. 3, c. 5), it is provided that no minor under the age of 15 years shall be enlisted in the naval service. Supplement to Rev. St. vol. 1, p. 484 [Q. S. Comp. St. 1901, pp. 1007, 1008]. During the session of Congress of 188
Passage 4:  to be 18 years and 7 months

In [12]:
# Test with a query
query = "What is the ruling on minors' ability to enlist in the navy?"
print(f"Query: {query}")

passages = retrieve_passages(query, top_k=5)
print("\nTop Retrieved Passages:")
for i, passage in enumerate(passages, 1):
    print(f"{i}. {passage}")

summary = generate_summary(query, passages)
print("\nSummary:", summary)

Query: What is the ruling on minors' ability to enlist in the navy?

Top Retrieved Passages:
1. ht of the parent, or other person irr loco parentis, to object to the enlistment of a minor in the navy, is clearly limited to cases where the minor is but 18 years of age. Jf he is above that age, and under 21 years of age, he can bind himself by enlisting until he attains his majority, at which ti
2. hority that enlistments in the navy, though, made without consent of the parent or guardian, are binding, and the minor cannot avoid them. See U. S. v. Bainbridge, 1 Mason, 71, Fed. Cas. No. 14,497; U. S. v. Blakeney, 3 Grat. 405.
But it is otherwise as to enlistments in the army. The distinction is
3.  statute since that time.
By an act of Congress approved May 12, 1879 (21 Stat. 3, c. 5), it is provided that no minor under the age of 15 years shall be enlisted in the naval service. Supplement to Rev. St. vol. 1, p. 484 [Q. S. Comp. St. 1901, pp. 1007, 1008]. During the session of Congress of

NameError: name 'torch' is not defined

In [10]:
print(f"FAISS index dimension: {index.d}")
print(f"Embedding model output dimension: {embedding_model.config.hidden_size}")

FAISS index dimension: 384
Embedding model output dimension: 384
